# Datasets
- [pageb_benchmarks.zip](https://ir.library.oregonstate.edu/concern/parent/47429f155/file_sets/1g05fh87w)


# 要求
使用[Python Outlier Detection (PyOD)](https://github.com/yzhao062/pyod)或其他已知的工具包来完成分析工作

# 提交的内容
- 完整的分析代码
- 分析报告：展示分析的思路，详细过程，结果及你的分析
- 所选择的数据集在README中说明，数据文件不要上传到Github中


In [1]:
import pandas as pd
import os
import time
import warnings
import numpy as np

warnings.filterwarnings('ignore')

# timekeeping
timekeeping = time.time()

In [2]:
PAGEB_ROOT = 'pageb/benchmarks'
benchmark_list = os.listdir(PAGEB_ROOT)
print(len(benchmark_list))

940


## 数据来源说明

根据论文[1]可知，数据集中会引入4种不同的层次的不相关特征（i.e., noise）。

要创建新的不相关特征，首先从原始母集中随机选择一个特征。 然后，对于原始数据集中的每个数据点，通过从原始数据点的值进行统一采样（替换）来为此特征选择一个值。 结果是新添加的特征与某些原始特征具有相同的边缘分布，但是其值不包含有关数据点异常状态的信息。这保留了真实数据的特质，同时允许引入噪声。

为了简化确定需要多少不相关特征的过程，如果数据集已经具有$d$维特征，而我们想评估$d^{'}$维，即将成对平均距离增加一个因子$\alpha$所需的维数，那么
\begin{equation}
d^{'} = \left( \alpha \sqrt{d}\right)^2 ~ ~ ~ ~(1)\,,
\end{equation}

其中$\alpha \in \{1.0, 1.2, 1.5, 2.0\}$.

[1] Emmott A, Das S, Dietterich T G, et al. A Meta-Analysis of the Anomaly Detection Problem[J]. arXiv: Artificial Intelligence, 2015.

随机选取一个csv文件，确定该数据集的原始特征有哪些？

In [3]:
df = pd.read_csv(os.path.join(PAGEB_ROOT, benchmark_list[0]))
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   point.id        30 non-null     object 
 1   motherset       30 non-null     object 
 2   origin          30 non-null     object 
 3   original.label  30 non-null     int64  
 4   diff.score      30 non-null     float64
 5   ground.truth    30 non-null     object 
 6   V               30 non-null     float64
 7   V.1             30 non-null     float64
 8   V.2             30 non-null     float64
 9   V.3             30 non-null     float64
 10  V.4             30 non-null     float64
 11  V.5             30 non-null     float64
 12  V.6             30 non-null     float64
 13  V.7             30 non-null     float64
 14  V.8             30 non-null     float64
 15  V.9             30 non-null     float64
dtypes: float64(11), int64(1), object(4)
memory usage: 3.9+ KB


,point.id,motherset,origin,original.label,diff.score,ground.truth,V,V.1,V.2,V.3,V.4,V.5,V.6,V.7,V.8,V.9
0,pageb_point_3544,pageb,multiclass,1,0.546693,nominal,-0.235923,-0.745876,-0.242177,-0.426234,0.272044,-1.183939,-0.017650,-0.280187,-0.386450,-0.625569
1,pageb_point_4573,pageb,multiclass,5,0.845264,anomaly,3.825138,2.060915,5.332560,-0.320025,-0.464919,-0.668298,0.019264,5.805619,9.252114,5.488889
2,pageb_point_1154,pageb,multiclass,1,0.151508,nominal,0.080523,0.300133,0.059718,-0.111419,-0.386159,0.491895,-0.061658,0.063817,0.293325,0.725230
3,pageb_point_4607,pageb,multiclass,5,0.844626,anomaly,3.825138,1.956314,5.127173,-0.324748,-0.487421,-0.972995,0.036925,5.500187,8.183820,4.371193
4,pageb_point_1248,pageb,multiclass,1,0.126937,nominal,-0.130441,0.831854,0.058068,0.305208,-0.583058,-1.019872,-0.061513,0.020522,0.087107,0.551898


根据以上的信息我们可以确定，pageb这个数据集的原始特征维度$d=10$(``V``, ``V.1``~``V.9``)。因此，由等式（1）可知，所有csv文件所包含的列数可能为$16=\left(1.0 \times \sqrt{10}\right)^2+6$, $20=\left(1.2 \times \sqrt{10}\right)^2+6$, $28=\left(1.5 \times \sqrt{10}\right)^2+6$, $46=\left(2.0 \times \sqrt{10}\right)^2+6$.

下面我们遍历所有csv文件，验证一下。

In [4]:
d_set = set()
d_count = 0
for i in range(len(benchmark_list)):
    df = pd.read_csv(os.path.join(PAGEB_ROOT, benchmark_list[i]))
    d_set.add(len(df.columns))
    d_count += len(df)
print('Possible columns of all csv files:', d_set)
print('Total amount:', d_count)

Possible columns of all csv files: {16, 20, 28, 46}
Total amount: 3180315


## 数据特征选择

为了充分利用所提供的数据集完成离群点分析与异常检测，将提取所有csv文件共同的特征（即原始特征,``V``, ``V.1``~``V.9``）作为算法或模型的输入，用于检测该条数据是否属于异常点。

In [5]:
ORIGIN_FEATURES = ['V', 'V.1', 'V.2', 'V.3', 'V.4', 'V.5', 'V.6', 'V.7', 'V.8', 'V.9', 'ground.truth']
def feature_section(benchmark_list):
    concat_data = pd.DataFrame()
    for i in benchmark_list:
        df = pd.read_csv(os.path.join(PAGEB_ROOT, i))
        concat_data = concat_data.append(df[ORIGIN_FEATURES])
    return concat_data

In [6]:
concat_data = feature_section(benchmark_list=benchmark_list)
concat_data.info()
concat_data.head()

KeyboardInterrupt: 

### 数据集划分
train set : test set = 8 : 2 

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(concat_data, test_size=0.2, random_state=2020)

def data_label_split(data, label_column='ground.truth'):
    x = data.drop(label_column, axis=1)
    y = []
    for i in data[label_column].values:
        if i == 'nominal':
            y.append(0)
        else:
            y.append(1)
    y = np.array(y)
    return x, y

X_train, y_train = data_label_split(train)
X_test, y_test = data_label_split(test)

In [ ]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

### t-SNE降维，用于可视化

In [ ]:
from sklearn.manifold import TSNE
# T-SNE Implementation
t0 = time.time()
X_train_reduced_tsne = TSNE(n_components=2, random_state=2020).fit_transform(X_train.values)
X_test_reduced_tsne = TSNE(n_components=2, random_state=2020).fit_transform(X_test.values)
t1 = time.time()
print("T-SNE took {:.2} s".format(t1 - t0))

## 模型比较
### 单一模型
- KNN
- PCA
- LOF

### 组合模型
- **Average**: average scores of all detectors
- **Maximization**: maximum score across all detectors.
- **Average of Maximum (AOM)**
- **Maximum of Average (MOA)**

ref: https://github.com/yzhao062/pyod/tree/master/examples

#### kNN
初始化一个 ``pyod.models.knn.KNN`` 检测器, 模型拟合, 然后给出预测。

In [ ]:
# train the KNN detector
from pyod.models.knn import KNN

clf_name = 'KNN'
clf = KNN()
clf.fit(X_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

利用 ``ROC`` 和 ``Precision @ Rank`` 评估预测。

In [ ]:
from pyod.utils.data import evaluate_print
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

可视化 ``KNN``的结果

In [ ]:
from pyod.utils.example import visualize
visualize(clf_name, X_train_reduced_tsne, y_train, X_test_reduced_tsne, y_test, y_train_pred,
          y_test_pred, show_figure=True, save_figure=False)

#### PCA
初始化一个 ``pyod.models.pca.PCA`` 检测器, 模型拟合, 然后给出预测。

In [ ]:
# train PCA detector
from pyod.models.pca import PCA

clf_name = 'PCA'
clf = PCA(n_components=3)
clf.fit(X_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

利用 ``ROC`` 和 ``Precision @ Rank`` 评估预测

In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

可视化 ``PCA`` 的结果

In [ ]:
visualize(clf_name, X_train_reduced_tsne, y_train, X_test_reduced_tsne, y_test, y_train_pred,
          y_test_pred, show_figure=True, save_figure=False)

#### LOF
初始化一个 ``pyod.models.lof.LOF`` 检测器, 模型拟合, 然后给出预测。

In [ ]:
# train LOF detector
from pyod.models.lof import LOF
clf_name = 'LOF'
clf = LOF()
clf.fit(X_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

利用 ``ROC`` 和 ``Precision @ Rank`` 评估预测

In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

可视化 ``LOF`` 的结果

In [ ]:
# visualize the results
visualize(clf_name, X_train_reduced_tsne, y_train, X_test_reduced_tsne, y_test, y_train_pred,
          y_test_pred, show_figure=True, save_figure=False)

#### Model Combination
用不同的k(10 ～ 200)初始化20个 ``kNN`` 离群点检测器，然后得到所有的离群点的分数。

In [ ]:
from pyod.models.knn import KNN  # kNN detector
from pyod.models.combination import aom, moa, average, maximization
from pyod.utils.utility import standardizer

In [ ]:
# standardizing data for processing
X_train_norm, X_test_norm = standardizer(X_train, X_test)

n_clf = 20  # number of base detectors

# initialize 20 base detectors for combination
k_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]

train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])

print('Combining {n_clf} kNN detectors'.format(n_clf=n_clf))

for i in range(n_clf):
    k = k_list[i]

    clf = KNN(n_neighbors=k, method='largest')
    clf.fit(X_train_norm)

    train_scores[:, i] = clf.decision_scores_
    test_scores[:, i] = clf.decision_function(X_test_norm)

In [ ]:
# Decision scores have to be normalized before combination
train_scores_norm, test_scores_norm = standardizer(train_scores, test_scores)

# Combination by average
y_by_average = average(test_scores_norm)
# Combination by max
y_by_maximization = maximization(test_scores_norm)
# Combination by aom
y_by_aom = aom(test_scores_norm, n_buckets=5)
# Combination by moa
y_by_moa = moa(test_scores_norm, n_buckets=5)

In [ ]:
print("\nOn Test Data:")
evaluate_print('Combination by Average', y_test, y_by_average)
evaluate_print('Combination by Maximization', y_test, y_by_maximization)
evaluate_print('Combination by AOM', y_test, y_by_aom)
evaluate_print('Combination by MOA', y_test, y_by_moa)

In [ ]:
m, s = divmod(time.time()-timekeeping, 60)
h, m = divmod(m, 60)
print ('run time: %02d:%02d:%02d' % (h, m, s))